# Uma análise quantitativa dos hotéis em Recife a partir dos dados do Trivago

Ângelo de Sant'Ana &mdash;
assd {em} cin.ufpe.br;

Antônio Rodrigues &mdash;
acpr {em} cin.ufpe.br;

Juliana Carvalho &mdash;
jco2 {em} cin.ufpe.br

junho de 2017

In [11]:
from bs4 import BeautifulSoup
import re

##### Localização dos dados relevantes

Abaixo começamos a localização dos elementos importantes das páginas .html para reconhecimento dos conjuntos que contêm
os valores que desejamos usar na nossa análise.

In [45]:
# fazer um iterador para capturar todas as páginas de 1 a 5

urlAvaliacoes = "pages/avaliacoesPag1.html"

parsed_page_avaliacoes = BeautifulSoup(open(urlAvaliacoes),"html.parser")

# para o nome do hotel
# class_="name__copytext m-0 item__slideout-toggle"

listaHoteis = []

nomesDosHoteisAvaliacoes = parsed_page_avaliacoes.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

divsAvaliacoes = parsed_page_avaliacoes.find_all('div', class_="rat-chart__group rat-chart__group--collapsed")

for temp in divsAvaliacoes:
    print(temp.text, "\n\n")

# print(nomesDosHoteisAvaliacoes)

# for hotel in nomesDosHoteisAvaliacoes:
#     listaHoteis.append(dict(zip(["nome", "localizacao", "quartos", "servico", "limpeza", "custo-beneficio", \
#                                  "conforto", "facilidades", "predio", "cafe-da-manha", "comida"], \
#                                 [hotel.text, , , , , , , , , , ])))

# print(divsAvaliacoes)

# para os conjuntos das avaliações
# class_="rat-chart__group rat-chart__group--collapsed"

# para o item de avaliação exibido
# class_="rat-chart__item"

# para o nome da característica (coluna)
# class_="rat-chart__label"

# para o valor da avaliação
# class_="parentesis-wrapper"


LocalizaçãoExcelente
92 / 100QuartosMuito boa84 / 100ServiçoBoa79 / 100LimpezaExcelente
85 / 100Custo-benefícioBoa78 / 100ConfortoExcelente
85 / 100FacilidadesMuito boa83 / 100PrédioMuito boa80 / 100Café da manhãNão avaliadoNão disponívelComidaBoa76 / 100 


LocalizaçãoExcelente
89 / 100QuartosMuito boa80 / 100ServiçoMuito boa84 / 100LimpezaMuito boa81 / 100Custo-benefícioBoa76 / 100ConfortoMuito boa81 / 100FacilidadesMuito boa80 / 100PrédioBoa75 / 100Café da manhãNão avaliadoNão disponívelComidaBoa77 / 100 


LocalizaçãoExcelente
86 / 100QuartosMuito boa82 / 100ServiçoBoa79 / 100LimpezaMuito boa84 / 100Custo-benefícioBoa77 / 100ConfortoMuito boa83 / 100FacilidadesMuito boa83 / 100PrédioBoa79 / 100Café da manhãAceitável70 / 100ComidaBoa75 / 100 


LocalizaçãoMuito boa84 / 100QuartosBoa77 / 100ServiçoMuito boa82 / 100LimpezaExcelente
86 / 100Custo-benefícioBoa77 / 100ConfortoMuito boa82 / 100FacilidadesMuito boa82 / 100PrédioMuito boa82 / 100Café da manhãBoa75 / 100ComidaBoa75 / 100 




In [ ]:
urlInfos = "pages/InfoPag1.html"

parsed_page_infos = BeautifulSoup(open(urlInfos,encoding="latin1"),"html.parser")

nomesDosHoteisInfos = parsed_page_infos.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

# para todas as comodidades do hotel
# class_="sl-box__block clearfix"

divsInfos = parsed_page_infos.find_all('div', class_="sl-box__block clearfix")

# print(divsInfos)

# para cada uma das comodidades do hotel
# class_="all-amenities__group mb-gutter"

# sobre os quartos
# class_="sl-box__content"